In [1]:
from models.lbanp_ar import LBANP_AR
from models.lbanp import LBANP
lbanp_ar_model = LBANP_AR(num_latents=8,
                dim_x= 1,
                dim_y= 1,
                d_model=64,
                emb_depth=4,
                dim_feedforward=128,
                nhead=4,
                dropout=0.0,
                num_layers=6)

lbanp_model = LBANP(num_latents=8,
                dim_x= 1,
                dim_y= 1,
                d_model=64,
                emb_depth=4,
                dim_feedforward=128,
                nhead=4,
                dropout=0.0,
                num_layers=6)

from data.gp import GPSampler, RBFKernel, Matern52Kernel
kernel = RBFKernel()
sampler = GPSampler(kernel)
batch = sampler.sample(batch_size=16,
            num_ctx=None,
            num_tar=None,
            max_num_points=50,
            x_range=(-2, 2),
            device='cpu')

In [3]:
import torch
from inference.ar_custom import ar_sample, ar_log_likelihood
xc = torch.randn(1,6,1)
yc = torch.randn(1,6,1)
xt = torch.randn(1,2,1)
lbanp_model.eval()
with torch.no_grad():
    lbanp_ar_model.reset_stored_qkv()
    y_sampled, y_noiseless, pred_dists_1 = ar_sample(lbanp_ar_model, xc, yc, xt, seed=1)
    lbanp_ar_model.reset_stored_qkv()
    y_mean, y_std, y_noiseless, pred_dists_2 = lbanp_ar_model.sample_ar(xc, yc, xt, num_samples=1, seed=1)

last k torch.Size([4, 8, 16]) tensor([[ 0.2056,  0.1267,  0.5569,  0.5546,  0.1739,  0.2853,  0.2501,  0.4325],
        [-0.3307, -0.3244,  0.0531,  0.1260,  0.4638, -0.1631, -0.2379, -0.4350],
        [-0.0741, -0.1856,  0.5242,  0.3534, -0.5014,  0.0902,  0.0799,  0.1067],
        [ 0.7219,  0.8307,  0.5443,  0.8657,  1.3877,  0.8846,  0.8199,  0.6449]])
now k torch.Size([4, 6, 16]) tensor([[ 0.0775,  0.2011,  0.1779,  0.2288,  0.3621,  0.1172],
        [-0.2707, -0.2641, -0.2644, -0.3820, -0.4865, -0.2039],
        [-0.3378, -0.0950, -0.2762,  0.0979,  0.0731, -0.2619],
        [ 0.9695,  0.9409,  0.8724,  0.6300,  0.6710,  0.9965]])
prev last_sim torch.Size([4, 8, 7]) tensor([[-1.6237e-01, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
         -3.4028e+38, -3.4028e+38],
        [-3.4028e+38, -4.2216e-01, -3.4028e+38, -3.4028e+38, -3.4028e+38,
         -3.4028e+38, -3.4028e+38],
        [-3.4028e+38, -3.4028e+38,  6.9993e-01, -3.4028e+38, -3.4028e+38,
         -3.4028e+38, -3.

In [ ]:
import timeit

lbanp_model.eval()

print("rollout=False")
execution_time = timeit.timeit(
    lambda: lbanp_model.predict(batch.xc, batch.yc, batch.xt, rollout=False),
    number=1000
)
print(f"Time taken: {execution_time:.4f} seconds")
pred_false = lbanp_model.predict(batch.xc, batch.yc, batch.xt, rollout=False)

print("\nrollout=True")
execution_time = timeit.timeit(
    lambda: lbanp_model.predict(batch.xc, batch.yc, batch.xt, rollout=True),
    number=1000
)
print(f"Time taken: {execution_time:.4f} seconds")
pred_true = lbanp_model.predict(batch.xc, batch.yc, batch.xt, rollout=True)

rollout=False
Time taken: 10.0140 seconds

rollout=True


In [7]:
import torch
lbanp_model.eval()
with torch.no_grad():
    pred = lbanp_ar_model.predict_ar(batch.xc, batch.yc, batch.xt, num_samples=1)

In [8]:
yt_mean, yt_std, yt_noiseless = pred

In [9]:
yt_noiseless.shape

torch.Size([1, 16, 26, 1])

context_encodings [tensor([[[ 2.0775e-01,  1.0336e+00,  1.8105e-01,  4.4963e-01,  1.2975e+00,
          -3.1780e-01, -3.9824e-01, -9.8852e-01,  5.8518e-01, -1.3709e+00,
          -3.6101e-01,  1.6496e+00,  6.8951e-01, -7.9364e-01,  9.1467e-01,
          -4.7548e-01,  1.1736e+00,  1.4644e-01, -2.0006e+00, -6.6503e-01,
          -9.2991e-01, -1.2588e+00,  1.3986e+00,  9.4222e-01, -1.3962e+00,
          -9.9916e-02, -1.7127e-01, -5.7860e-01, -3.0937e-01,  4.3780e-02,
           5.6832e-01,  9.3864e-01,  7.6166e-01,  4.4027e-01, -4.0391e-01,
           1.3908e+00,  1.0744e+00,  4.2628e-01, -9.8801e-01,  4.5836e-01,
           5.7359e-01,  2.0829e-02, -1.0609e+00,  7.5202e-01,  2.7238e+00,
           8.5937e-01, -1.1415e+00,  4.7777e-02,  1.1698e-01, -1.2541e+00,
          -2.3130e-01, -9.6826e-02,  1.1858e+00, -1.3183e+00,  4.3670e-01,
           9.5399e-01, -8.6321e-01,  7.0113e-03,  7.9244e-02, -1.6692e+00,
           2.9624e-02, -1.1310e+00,  1.2450e-01, -2.7109e+00],
         [-5.7639e

In [8]:
pred_dists_1

[Normal(loc: tensor([[[-0.0033]]]), scale: tensor([[[0.7373]]])),
 Normal(loc: tensor([[[0.0203]]]), scale: tensor([[[0.7291]]]))]

In [9]:
pred_dists_2

[Normal(loc: tensor([[[-0.0033]]]), scale: tensor([[[0.7373]]])),
 Normal(loc: tensor([[[0.0143]]]), scale: tensor([[[0.7296]]]))]

In [3]:

# import yaml
# import os.path as osp
# import torch

# name = "lbanp"
# device = "cpu"


# # lbanp_model = LBANP(num_latents=8,
# #                 dim_x= 2,
# #                 dim_y= 5,
# #                 d_model=16,
# #                 emb_depth=3,
# #                 dim_feedforward=64,
# #                 nhead=1,
# #                 dropout=0.0,
# #                 num_layers=4,
# #                 drop_y=0.5,
# #                 device="cpu")

# with open(osp.join("configs", "gp", f"{name}.yaml")) as g:
#     config = yaml.safe_load(g)

# path = osp.join("results", "gp", "lbanp", "lbanp-num_latents-8")
# filename = 'ckpt.tar'

# model = LBANP(**config).to(device)
# model.device = torch.device(device)
# model.eval()
# file = osp.join(path, filename)
# if not osp.exists(file):
#     raise FileNotFoundError(file)
# else:
#     ckpt = torch.load(file)
#     model.load_state_dict(ckpt.model)
# # tnp_model = TNPND(
# #         dim_x= 2,
# #                 dim_y= 5,
# #                 d_model=16,
# #                 emb_depth=3,
# #                 dim_feedforward=64,
# #                 nhead=1,
# #                 dropout=0.0,
# #                 num_layers=4,
# #                 num_std_layers=2,
# #                 drop_y=0.5,
# #                 cov_approx='lowrank', # cholesky or lowrank parameterization
# #                 prj_dim=20,
# #                 prj_depth=4,
# #                 diag_depth=4,
# #     device="cpu"
# #     )

In [8]:
pred.loc

tensor([[[ 0.0865],
         [ 0.1216],
         [ 0.0579],
         [ 0.1175],
         [ 0.0976]],

        [[ 0.0677],
         [ 0.0694],
         [ 0.1275],
         [ 0.0643],
         [ 0.0389]],

        [[ 0.1122],
         [ 0.0924],
         [ 0.1512],
         [ 0.1329],
         [ 0.1409]],

        [[ 0.0946],
         [ 0.1158],
         [ 0.0810],
         [ 0.0968],
         [ 0.1216]],

        [[ 0.0881],
         [ 0.1125],
         [ 0.0663],
         [ 0.1328],
         [ 0.0951]],

        [[ 0.0932],
         [ 0.0515],
         [ 0.0839],
         [ 0.0934],
         [ 0.0877]],

        [[ 0.0948],
         [ 0.0667],
         [ 0.0224],
         [ 0.0649],
         [ 0.0203]],

        [[ 0.0435],
         [ 0.1192],
         [ 0.0921],
         [ 0.1208],
         [ 0.1000]],

        [[ 0.1036],
         [ 0.0800],
         [ 0.0583],
         [ 0.1249],
         [ 0.0879]],

        [[ 0.0799],
         [ 0.1329],
         [ 0.0743],
         [ 0.0783],
  

In [9]:
ar_pred = model.ar_predict(batch.xc, batch.yc, batch.xt)

In [10]:
ar_pred.loc

tensor([[[-4.9805e-01],
         [-2.1439e-01],
         [-7.0436e-02],
         [-3.8150e-01],
         [ 1.3432e-01],
         [-2.1759e-01]],

        [[-1.0552e+00],
         [-1.4019e+00],
         [-1.0560e+00],
         [ 1.5774e+00],
         [-8.5855e-01],
         [ 1.4868e+00]],

        [[-1.0756e-01],
         [ 9.7629e-01],
         [ 1.6945e+00],
         [ 1.6219e+00],
         [-1.5045e-01],
         [-8.5618e-01]],

        [[-1.6472e-01],
         [ 6.9761e-02],
         [ 1.4657e-01],
         [ 2.6029e-01],
         [-2.1780e-01],
         [ 4.2241e-01]],

        [[ 3.1720e-01],
         [-1.3526e-01],
         [ 3.2057e-01],
         [ 3.3987e-01],
         [ 3.2335e-01],
         [-4.9626e-02]],

        [[ 3.5101e-01],
         [-7.4132e-06],
         [ 1.1210e+00],
         [ 5.4672e-01],
         [ 7.0842e-01],
         [ 1.0338e+00]],

        [[-1.7356e-01],
         [ 8.7545e-02],
         [-1.5150e-01],
         [-1.8290e-01],
         [-3.6729e-03],
    

In [11]:
model.latents

Parameter containing:
tensor([[-5.2829e-01, -6.5334e-01,  3.0985e+00, -1.3711e+00, -1.7328e+00,
         -5.9612e-01,  4.6291e-01, -5.1603e-01, -8.6992e-01,  5.4297e-02,
          1.9330e-01, -4.9053e-01,  9.7111e-01,  2.5913e+00,  1.6435e+00,
          6.1639e-01,  1.4586e+00, -2.2873e+00,  8.3987e-01,  3.4112e-01,
         -9.9662e-01, -3.1962e-01, -6.2789e-01, -1.5592e+00, -3.2935e-01,
         -4.9620e-02,  8.0759e-02,  5.3290e-01, -3.0790e-01,  2.7566e-01,
         -2.1659e-01,  4.6147e-01, -5.2832e-01, -8.6208e-01, -1.8639e-01,
          7.3401e-01,  1.8744e+00, -4.3843e-01, -1.3752e+00,  4.8039e-01,
         -3.8753e-01,  5.9233e-01,  2.5546e-01,  3.6070e+00,  1.0225e+00,
          1.4219e+00, -6.2496e-02, -1.3350e+00, -6.3257e-02,  8.0941e-01,
         -6.2201e-01,  2.5051e-01, -5.2361e-01,  2.8048e+00,  1.8173e-01,
         -2.4239e-01, -3.1172e-01,  1.9425e+00,  8.4579e-01, -1.1546e+00,
          7.8763e-01, -3.4105e-01,  1.5788e-01,  2.2112e-01],
        [ 1.0704e-01, -2.853

In [12]:
batch.yc.shape

torch.Size([8, 512, 5])

In [13]:
batch.xt.shape

torch.Size([8, 50, 2])

In [14]:
batch.yt.shape

torch.Size([8, 50, 5])

In [7]:
ll_loss, pred = lbanp_model(batch)

In [14]:
tnp_model(batch)

AttrDict({'loss_target': tensor(666.4536, grad_fn=<NegBackward0>), 'loss_ctx': tensor(1781.4780, grad_fn=<MeanBackward0>), 'loss': tensor(2447.9316, grad_fn=<AddBackward0>), 'mean_std': tensor(0.0412, grad_fn=<MeanBackward0>), 'rmse': tensor(336.7043, grad_fn=<MeanBackward0>)})

In [17]:
pred.sample(torch.Size([10]))[1]

tensor([[[-0.4370, -1.0098,  0.1075, -0.9991,  0.4572],
         [-0.1563, -0.3379,  0.4788,  0.1298,  0.6714],
         [ 0.2853, -0.2805,  0.3360,  0.8649, -0.9022],
         ...,
         [ 1.2777,  0.1652,  0.6480,  0.2400,  1.1537],
         [ 0.7377,  0.2597,  1.0902, -0.0205, -0.4204],
         [ 0.1903, -0.3400,  0.1030, -0.1797,  0.2685]],

        [[-0.3725,  0.3566, -1.8452,  0.7515,  0.4054],
         [ 0.6287, -0.5361,  1.3953,  1.1158, -0.8499],
         [-0.5555, -0.6780,  3.5492,  0.5310,  1.0871],
         ...,
         [ 1.2121,  0.1738,  2.3070, -1.4268,  1.1481],
         [ 1.1972, -0.1266,  0.5311, -0.2256,  0.8538],
         [ 1.8189, -0.7754, -0.6902, -0.3359, -0.3103]],

        [[ 0.8968,  0.4521,  0.4219, -1.3565, -0.3342],
         [-1.7084,  0.4685,  1.7808,  0.9149,  0.6502],
         [ 1.3056, -1.0105, -0.4376,  0.3604,  2.1575],
         ...,
         [ 0.1437, -1.3362, -0.9561,  0.4158,  0.5472],
         [ 1.8388, -0.7121, -1.5469, -0.1130, -0.5621],
  

In [23]:
batch.yc.shape

torch.Size([8, 512, 5])

In [8]:
import numpy as np
a = np.random.randn(3,2,2)

In [9]:
a

array([[[ 0.21669559, -0.74235375],
        [-0.16376122, -0.70650212]],

       [[-0.53418005,  0.00455652],
        [ 0.57478282,  0.07178991]],

       [[ 0.75321925,  0.30662907],
        [ 0.62068709,  0.79254104]]])

In [10]:
a.reshape(-1,2).reshape(3,2,2)

array([[[ 0.21669559, -0.74235375],
        [-0.16376122, -0.70650212]],

       [[-0.53418005,  0.00455652],
        [ 0.57478282,  0.07178991]],

       [[ 0.75321925,  0.30662907],
        [ 0.62068709,  0.79254104]]])